In [ ]:
import datetime
import json
data = json.load(
    open(
        "data/tingle-pilot-collected-data-export.json"
    )
)

In [40]:
print([datetime.datetime.fromtimestamp(int(k)/1000) for k in [data["samples"][j] for j in data["samples"]][0]])

[datetime.datetime(2018, 1, 19, 17, 14, 9, 404000), datetime.datetime(2018, 1, 19, 17, 14, 11, 54000), datetime.datetime(2018, 1, 19, 17, 14, 13, 4000), datetime.datetime(2018, 1, 19, 17, 14, 14, 954000), datetime.datetime(2018, 1, 19, 17, 14, 17, 54000), datetime.datetime(2018, 1, 19, 17, 14, 18, 855000), datetime.datetime(2018, 1, 19, 17, 14, 20, 654000), datetime.datetime(2018, 1, 19, 17, 14, 22, 604000), datetime.datetime(2018, 1, 19, 17, 14, 24, 405000), datetime.datetime(2018, 1, 19, 17, 14, 26, 654000), datetime.datetime(2018, 1, 19, 17, 14, 28, 305000), datetime.datetime(2018, 1, 19, 17, 14, 30, 405000), datetime.datetime(2018, 1, 19, 17, 14, 32, 54000), datetime.datetime(2018, 1, 19, 17, 14, 34, 154000), datetime.datetime(2018, 1, 19, 17, 14, 35, 804000), datetime.datetime(2018, 1, 19, 17, 14, 37, 754000), datetime.datetime(2018, 1, 19, 17, 14, 39, 404000), datetime.datetime(2018, 1, 19, 17, 14, 41, 205000), datetime.datetime(2018, 1, 19, 17, 14, 43, 4000), datetime.datetime(2

---

In [54]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [56]:
%%cython --annotate

cdef extern from "gesture-recognition-project/software/firmware/nrf52_X9_TRICH3/neural_networks_LHAND_LHEAD.h":
    
    float nn_lefthand_lefthead_5521(float t1, float t2, float t3, float  t4, float distance)
    
    float nn_lefthand_lefthead_7521(float t1, float t2, float t3, float  t4, float distance, float pitch, float roll)

CompileError: command 'gcc' failed with exit status 1

CompileError: command 'gcc' failed with exit status 1